Student Name: Joel Amarou Heuer

Student ID: 202102201

# Imports

In [11]:
import numpy as np
import pandas as pd
import random

# (Hyper)Parameters

In [12]:
# TODO Hyperparameters
# input_size = 
output_size = 10 # -> number of neurons in output-layer (= for each possible rating?)
# [hidden_layers_sizes] = 
# learning_rate = 
number_of_epochs = 1 # how is every sample used for pass/forward/backward

# Parameters
train_data_path = "./data/drugLibTrain_raw.tsv" # please use relative path like this
test_data_path = "./data/drugLibTest_raw.tsv" # please use relative path like this

In [4]:
'''
EXAMPLE

inputs = [1,2,3,4]    das sind 4 Eingaben, also entweder 
						-> 4 Features für Input-Layer oder 
						-> 4 Neuronen der davorigen Layer

weights = [
			[w11,w12,w13,w14]       -> Neuoron1 hat 4 Weights, also von jedem Neuron der davorigen Schicht eins
			[w21,w22,w23,w24]		-> Neuoron2 hat 4 Weights, also von jedem Neuron der davorigen Schicht eins
		]
	  2x4
	#NeuronenProHiddenLayer x #Inputs
'''



def forward_pass(data):
	'''
	@param data is comment/review in string format
	-> for a string s, one-hot-encoding makes matrix with dimensions:     {#words-in-string}         x     {#all-words-in-dataset = input-size}

	multiply with weights-matrix, which has dimensions:				:     {#neuron-in-hidden-layeri} x     {#input-size}


	Formula1: 	weights     *      inputs       +     biases
	       	    {#n/l}x{#ip}       {#ip}{??}          {#ip}{??}


	

	
	'''

	print(data)
	# LayerInput --> LayerHidden1


	# LayerHidden2 --> LayerHidden2

	# LayerHidden2 --> LayerOutput
	pass


def train(train_data, train_labels, valid_data, valid_labels):
	for epoch in range(number_of_epochs):
		index = 0

		# Same thing about [hidden_layers] mentioned above is valid here also
		for data, labels in zip(train_data, train_labels):
			predictions  = forward_pass(data)
			break
			#loss_signals = derivation_of_loss_function(labels, predictions)
			#backward_pass(data, [hidden_layers], predictions, loss_signals)
			#loss = loss_function(labels, predictions)

			#if index % 2000 == 0: # at each 2000th sample, we run validation set to see our model's improvements
			#	accuracy, loss = test(valid_data, valid_labels)
			#	print("Epoch= "+str(epoch)+", Coverage= %"+ str(100*(index/len(train_data))) + ", Accuracy= "+ str(accuracy) + ", Loss= " + str(loss))
#
			#index += 1

#   return losses

# Main

In [69]:
# Load Data
input_features =  'commentsReview'

train_data = pd.read_csv(train_data_path, sep='\t')
test_data = pd.read_csv(test_data_path, sep='\t')

# drop nans
train_data.dropna(subset = [input_features], inplace=True)
test_data.dropna(subset = [input_features], inplace=True)


# split feature and y columns
train_x = train_data[input_features]
train_y = train_data['rating']
test_x = test_data[input_features]
test_y = test_data['rating']


# ONE-HOT for classes 1-10
# creating one-hot vector notation of labels. (Labels are given numeric in the dataset)
new_train_y = np.zeros(shape=(len(train_y), output_size))
new_test_y = np.zeros(shape=(len(test_y), output_size))

# add 1's into one-hot-matrix
for i in range(len(train_y)) :    
    new_train_y[i][train_y[i]-1] = 1

for i in range(len(test_y)):
    new_test_y[i][test_y[i]-1] = 1

train_y = new_train_y
test_y = new_test_y

# Split training data into [a] Training (75%) and [b] validation (25%)
valid_x = np.asarray(train_x[int(0.75*len(train_x)):-1])
valid_y = np.asarray(train_y[int(0.75*len(train_y)):-1])
train_x = np.asarray(train_x[0:int(0.75*len(train_x))])
train_y = np.asarray(train_y[0:int(0.75*len(train_y))])

# TODO
train(train_x, train_y, valid_x, valid_y)
#print("Test Scores:")
#print(test(test_x, test_y))

KeyError: 147

In [81]:
import nltk
nltk.download('punkt')


train_data = pd.read_csv(train_data_path, sep='\t')
test_data = pd.read_csv(test_data_path, sep='\t')

df_data =  train_data.append(test_data)

df_data["commentsReview"].isnull().values.sum()
df_data.dropna(subset = ["commentsReview"], inplace=True)


dictonary_words = df_data["commentsReview"].apply(nltk.word_tokenize)
dictonary_words = dictonary_words.values.tolist()
# flat lists
dictonary_words = [item for sublist in dictonary_words for item in sublist]
dictonary_words = list(set(dictonary_words))
len(dictonary_words)

[nltk_data] Downloading package punkt to /home/korosu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/tmp/ipykernel_9813/2729530844.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_data =  train_data.append(test_data)


13480